# Library

In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

# File Load
- 4등급 등록&제원(말소제거)
- 정기
- 정밀

In [4]:
ods_fold = 'D:/data/big2/ODS'

In [9]:
# about 24s
file_name = '[ODS][G4]등록정보&제원정보(말소제거)(2022.12.29).csv'
df = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)

In [10]:
# about 
file_name = '[ODS]정기검사(2022.12.29).csv'
ins = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)

In [11]:
# about 
file_name = '[ODS]정밀검사(2022.12.29).csv'
dins = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)

In [12]:
df.shape, ins.shape, dins.shape

((1511251, 49), (22006426, 39), (28682228, 41))

In [13]:
df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'],
      dtype='object')

In [14]:
ins.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간'],
      dtype='object')

In [15]:
dins.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간',
       '검사 검사소코드', '검사유효기간'],
      dtype='object')

# Preprocessing

## 전처리 : 등록&제원
- 중복 차대번호 처리 : '최초등록일자', '배출가스검사유효일자’, 내림차순 정렬(ascending=[False, False])
- 배출가스등급 : 4등급
- 연료 : 경유차

In [25]:
df.shape

(1511251, 49)

In [26]:
df = df.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
df.shape

(1511211, 49)

In [27]:
df['배출가스등급'].value_counts(dropna=False)

4     1511174
T4         37
Name: 배출가스등급, dtype: int64

In [28]:
df['연료'].value_counts(dropna=False)

경유             1152697
휘발유             332313
LGP(액화석유가스)      26089
NaN                 71
기타연료                39
CNG(압축천연가스)          1
알코올                  1
Name: 연료, dtype: int64

In [29]:
df.shape

(1511211, 49)

In [30]:
df = df[df['연료'] == '경유'].reset_index(drop=True)
df.shape

(1152697, 49)

## 전처리 : 정기, 정밀 검사
- 정기, 정밀 검사 필요한 컬럼만 추출
- 정기, 정밀 검사 샘플에 '정기, 정밀' 검사 표기
- 정기, 정밀 통합하여 중복 차대번호 제거(최신 '검사일자'만 사용)

In [17]:
len(ins.columns), len(dins.columns)

(39, 41)

In [18]:
col = ['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6',]
len(col)

33

In [19]:
# about 30s
ins = ins[col]
dins = dins[col]
ins.shape, dins.shape

((22006426, 33), (28682228, 33))

In [20]:
ins['정기정밀'] = '정기'
dins['정기정밀'] = '정밀'

In [21]:
# about 13s
ins.shape, len(ins['차대번호'].unique())

((22006426, 34), 12601262)

In [32]:
# 20s
dins.shape, len(dins['차대번호'].unique())

((28682228, 34), 15367596)

In [33]:
# about 
tin = pd.concat([ins, dins], ignore_index=True)
tin.shape

(50688654, 34)

In [34]:
tin.shape[0] == ins.shape[0] + dins.shape[0]

True

In [35]:
tin = tin.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
tin.shape

(22044208, 34)

## 전처리 : 등록&제원과 정기,정밀 병합
- 4등급 경유 등록&제원 : df
- 최신 검사일자 중복 차대번호 제거 정기, 정밀 : tin
- 병합 : total

In [36]:
# about 45s
total = df.merge(tin, on='차대번호', how='left')
total.shape

(1152697, 82)

# Analysis
- '배출가스인증번호'별 '무부하매연판정1'별 '차대번호' 개수
- '배출가스인증번호'별 '무부하매연판정1'별 '무부하매연측정치1'의 통계

In [41]:
total['무부하매연판정1'].value_counts(dropna=False)

Y      1085469
N        34667
NaN      32561
Name: 무부하매연판정1, dtype: int64

In [60]:
total['무부하매연판정1'].isnull().sum()

32561

In [42]:
total['무부하매연측정치1'].value_counts(dropna=False)

0.00      282188
1.00       84819
2.00       70060
3.00       62826
4.00       62018
           ...  
135.00         1
2.50           1
103.00         1
0.01           1
0.03           1
Name: 무부하매연측정치1, Length: 111, dtype: int64

In [59]:
total['무부하매연측정치1'].isnull().sum()

32556

In [44]:
total.groupby(['무부하매연판정1'], dropna=False)['차대번호'].agg(['count'])

,count
무부하매연판정1,
N,34667
Y,1085469
NaN,32561


In [64]:
total.groupby(['무부하매연판정1'], dropna=False)['차대번호'].agg(['count'])

,count
무부하매연판정1,
N,34667
Y,1085469
NaN,32561


In [61]:
gas_grp = total.groupby(['배출가스인증번호', '무부하매연판정1'], dropna=False)['무부하매연측정치1'].agg(['count', 'mean', 'min', 'max'])
gas_grp

count       mean   min   max
배출가스인증번호     무부하매연판정1                              
4MY-AD-20    N             2  52.500000  51.0  54.0
             Y            30  12.233333   0.0  31.0
             NaN           0        NaN   NaN   NaN
4MY-DC-33    Y             6   6.000000   0.0  20.0
             NaN           0        NaN   NaN   NaN
...                      ...        ...   ...   ...
FMY-VTK-14-3 NaN           0        NaN   NaN   NaN
FMY-VTK-14-4 Y            49   0.918367   0.0  21.0
NaN          N           114  36.456140  10.0  94.0
             Y          6981   3.485178   0.0  41.0
             NaN           0        NaN   NaN   NaN

[451 rows x 4 columns]

In [73]:
gas_vin_grp = total.groupby(['배출가스인증번호', '무부하매연판정1'], dropna=False)['차대번호'].agg(['count'])
gas_vin_grp

count
배출가스인증번호     무부하매연판정1       
4MY-AD-20    N             2
             Y            30
             NaN           2
4MY-DC-33    Y             6
             NaN           1
...                      ...
FMY-VTK-14-3 NaN           2
FMY-VTK-14-4 Y            49
NaN          N           114
             Y          6981
             NaN         525

[451 rows x 1 columns]

In [74]:
gas_grp_y = total[total['무부하매연판정1'] == 'Y'].groupby(['무부하매연판정1', '배출가스인증번호'], dropna=False)['무부하매연측정치1'].agg(['count', 'mean', 'min', 'max'])
gas_grp_y

count       mean  min   max
무부하매연판정1 배출가스인증번호                                 
Y        4MY-AD-20        30  12.233333  0.0  31.0
         4MY-DC-33         6   6.000000  0.0  20.0
         4MY-PA-06        12  14.500000  0.0  40.0
         5MY-AD-13-06    336   1.247024  0.0  41.0
         5MY-AD-13-07    110   1.781818  0.0  45.0
...                      ...        ...  ...   ...
         FMY-VTK-14-1   1757   0.892999 -1.0  19.0
         FMY-VTK-14-2     86   0.569767  0.0  10.0
         FMY-VTK-14-3     74   0.202703  0.0   6.0
         FMY-VTK-14-4     49   0.918367  0.0  21.0
         NaN            6981   3.485178  0.0  41.0

[204 rows x 4 columns]

In [75]:
gas_grp_y_vin = total[total['무부하매연판정1'] == 'Y'].groupby(['무부하매연판정1', '배출가스인증번호'], dropna=False)['차대번호'].agg('count')
gas_grp_y_vin

무부하매연판정1  배출가스인증번호    
Y         4MY-AD-20         30
          4MY-DC-33          6
          4MY-PA-06         12
          5MY-AD-13-06     336
          5MY-AD-13-07     110
                          ... 
          FMY-VTK-14-1    1757
          FMY-VTK-14-2      86
          FMY-VTK-14-3      74
          FMY-VTK-14-4      49
          NaN             6981
Name: 차대번호, Length: 204, dtype: int64

In [76]:
gas_grp_n = total[total['무부하매연판정1'] == 'N'].groupby(['무부하매연판정1', '배출가스인증번호'], dropna=False)['무부하매연측정치1'].agg(['count', 'mean', 'min', 'max'])
gas_grp_n

count       mean   min   max
무부하매연판정1 배출가스인증번호                                  
N        4MY-AD-20         2  52.500000  51.0  54.0
         5MY-AD-13-06      1  51.000000  51.0  51.0
         5MY-AD-14-08      1  65.000000  65.0  65.0
         5MY-AD-14-18      4  35.000000  20.0  49.0
         5MY-HB-24-02      5  48.600000  30.0  80.0
...                      ...        ...   ...   ...
         9MY-KM-14-41   2293  41.387266  16.0  99.0
         9MY-PA-13-1       1  22.000000  22.0  22.0
         C135              2  56.000000  53.0  59.0
         FMY-VTK-14-1      4  24.750000  18.0  38.0
         NaN             114  36.456140  10.0  94.0

[104 rows x 4 columns]

In [77]:
gas_grp_n_vin = total[total['무부하매연판정1'] == 'N'].groupby(['무부하매연판정1', '배출가스인증번호'], dropna=False)['차대번호'].agg('count')
gas_grp_n_vin

무부하매연판정1  배출가스인증번호    
N         4MY-AD-20          2
          5MY-AD-13-06       1
          5MY-AD-14-08       1
          5MY-AD-14-18       4
          5MY-HB-24-02       5
                          ... 
          9MY-KM-14-41    2293
          9MY-PA-13-1        1
          C135               2
          FMY-VTK-14-1       4
          NaN              114
Name: 차대번호, Length: 104, dtype: int64

In [81]:
gas_grp_y.reset_index()

,무부하매연판정1,배출가스인증번호,count,mean,min,max
0,Y,4MY-AD-20,30,12.233333,0.0,31.0
1,Y,4MY-DC-33,6,6.000000,0.0,20.0
2,Y,4MY-PA-06,12,14.500000,0.0,40.0
3,Y,5MY-AD-13-06,336,1.247024,0.0,41.0
4,Y,5MY-AD-13-07,110,1.781818,0.0,45.0
...,...,...,...,...,...,...
199,Y,FMY-VTK-14-1,1757,0.892999,-1.0,19.0
200,Y,FMY-VTK-14-2,86,0.569767,0.0,10.0
201,Y,FMY-VTK-14-3,74,0.202703,0.0,6.0
202,Y,FMY-VTK-14-4,49,0.918367,0.0,21.0


In [82]:
gas_grp_n.reset_index()

,무부하매연판정1,배출가스인증번호,count,mean,min,max
0,N,4MY-AD-20,2,52.500000,51.0,54.0
1,N,5MY-AD-13-06,1,51.000000,51.0,51.0
2,N,5MY-AD-14-08,1,65.000000,65.0,65.0
3,N,5MY-AD-14-18,4,35.000000,20.0,49.0
4,N,5MY-HB-24-02,5,48.600000,30.0,80.0
...,...,...,...,...,...,...
99,N,9MY-KM-14-41,2293,41.387266,16.0,99.0
100,N,9MY-PA-13-1,1,22.000000,22.0,22.0
101,N,C135,2,56.000000,53.0,59.0
102,N,FMY-VTK-14-1,4,24.750000,18.0,38.0


In [83]:
pd.merge(gas_grp_y.reset_index(), gas_grp_n.reset_index(), on='배출가스인증번호', how='outer')

,무부하매연판정1_x,배출가스인증번호,count_x,mean_x,min_x,max_x,무부하매연판정1_y,count_y,mean_y,min_y,max_y
0,Y,4MY-AD-20,30.0,12.233333,0.0,31.0,N,2.0,52.50000,51.0,54.0
1,Y,4MY-DC-33,6.0,6.000000,0.0,20.0,NaN,NaN,NaN,NaN,NaN
2,Y,4MY-PA-06,12.0,14.500000,0.0,40.0,NaN,NaN,NaN,NaN,NaN
3,Y,5MY-AD-13-06,336.0,1.247024,0.0,41.0,N,1.0,51.00000,51.0,51.0
4,Y,5MY-AD-13-07,110.0,1.781818,0.0,45.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
200,Y,FMY-VTK-14-2,86.0,0.569767,0.0,10.0,NaN,NaN,NaN,NaN,NaN
201,Y,FMY-VTK-14-3,74.0,0.202703,0.0,6.0,NaN,NaN,NaN,NaN,NaN
202,Y,FMY-VTK-14-4,49.0,0.918367,0.0,21.0,NaN,NaN,NaN,NaN,NaN
203,Y,NaN,6981.0,3.485178,0.0,41.0,N,114.0,36.45614,10.0,94.0


In [92]:
with pd.ExcelWriter('analysis2/4등급_데이터_점검/[BD1][통계][G4]배인번호별_매연측정치_통계.xlsx') as writer:
    # 전체 합격/불합격별 대수 통계
    total.groupby(['무부하매연판정1'], dropna=False)['차대번호'].agg(['count']).to_excel(writer, sheet_name='판정별_통계')
    # 배출가스인증번호, 판정별 매연측정치 통계
    gas_grp.reset_index().to_excel(writer, sheet_name = '매연측정치_통계', index=False)
    gas_vin_grp.reset_index().to_excel(writer, sheet_name = '매연측정치_차대번호_통계', index=False)
    # 매연판정 합격/불합격별 매연측정치 통계
    gas_grp_y.reset_index().to_excel(writer, sheet_name='합격_매연측정치_통계', index=False)
    gas_grp_y_vin.reset_index().to_excel(writer, sheet_name='합격_매연측정치_차대번호_통계', index=False)
    gas_grp_n.reset_index().to_excel(writer, sheet_name='불합격_매연측정치_통계', index=False)
    gas_grp_n_vin.reset_index().to_excel(writer, sheet_name='불합격_매연측정치_차대번호_통계', index=False)
    # 합격/불합격 매연측정치 비교 통계
    pd.merge(gas_grp_y.reset_index(), gas_grp_n.reset_index(), on='배출가스인증번호', how='outer').to_excel(writer, sheet_name='합격_불합격_매연측정치_통계_비교', index=False)

In [89]:
gas_grp.reset_index().to_excel('analysis2/4등급_데이터_점검/[BD1][통계][G4]배인번호별_매연측정치_통계(test).xlsx', index=False)